# MPC Part 2- TP2 : Feature selection and non-linear regression 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# Load the data
ozone = pd.read_csv('ozone.txt', sep = ' ')
ozone_new = pd.read_csv('ozone_n.txt', sep = ' ')
ozone = pd.concat([ozone, ozone_new], ignore_index=True)
ozone
# Same dataset than TP1

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
0,87,15.6,18.5,18.4,4,4,8,0.6946,-1.7101,-0.6946,84
1,82,17.0,18.4,17.7,5,5,7,-4.3301,-4.0000,-3.0000,87
2,92,15.3,17.6,19.5,2,5,4,2.9544,1.8794,0.5209,82
3,114,16.2,19.7,22.5,1,1,0,0.9848,0.3473,-0.1736,92
4,94,17.4,20.5,20.4,8,8,7,-0.5000,-2.9544,-4.3301,114
...,...,...,...,...,...,...,...,...,...,...,...
106,59,18.3,18.3,19.0,7,7,7,-3.9392,-1.9284,-1.7101,66
107,70,17.1,18.2,18.0,7,7,7,-4.3301,-7.8785,-5.1962,72
108,81,19.6,25.1,27.2,3,4,4,-1.9284,-2.5712,-4.3301,57
109,146,27.0,32.7,33.7,0,0,0,2.9544,6.5778,4.3301,121


In [2]:
# Train / Test split:
from sklearn.model_selection import train_test_split
train , test = train_test_split(ozone, test_size = 0.20, random_state = 2) 
# to have the same split than me

## Variable selection

In [4]:
import numpy as np

import statsmodels.api as sm

def step_selection_adj(train, v_s, v_nu, idx_t):
    best_adj_r2 = -np.inf
    best_variable = None

    for var in v_nu:
        # Combine selected variables with the current variable
        current_vars = np.append(v_s, var)
        
        # Prepare the data for regression
        X = train.iloc[:, current_vars]
        y = train.iloc[:, idx_t]
        X = sm.add_constant(X)  # Add constant for intercept
        
        # Fit the model
        model = sm.OLS(y, X).fit()
        
        # Check adjusted R^2
        if model.rsquared_adj > best_adj_r2:
            best_adj_r2 = model.rsquared_adj
            best_variable = var

    return best_variable

In [5]:
def forward_selection_adj(train, idx_p, idx_t):
    selected_vars = []  # List to store selected variables
    non_selected_vars = idx_p.copy()  # Variables available for selection
    best_adj_r2 = -np.inf  # Initialize the best adjusted R^2
    stop = False  # Stopping criterion

    while non_selected_vars and not stop:
        # Perform one step of forward selection
        best_var = step_selection_adj(train, selected_vars, non_selected_vars, idx_t)
        
        # Combine selected variables with the best variable
        current_vars = np.append(selected_vars, best_var)
        
        # Prepare the data for regression
        X = train.iloc[:, current_vars]
        y = train.iloc[:, idx_t]
        X = sm.add_constant(X)  # Add constant for intercept
        
        # Fit the model
        model = sm.OLS(y, X).fit()
        
        # Check if adjusted R^2 improves
        if model.rsquared_adj > best_adj_r2:
            best_adj_r2 = model.rsquared_adj
            selected_vars.append(best_var)
            non_selected_vars.remove(best_var)
        else:
            stop = True  # Stop if performance decreases

    return selected_vars

In [6]:
from sklearn.metrics import mean_squared_error

# Define predictive variables (all columns except the target 'y')
idx_p = list(range(1, ozone.shape[1]))  # Columns x1 to x10
idx_t = 0  # Target column 'y'

# Apply forward selection to find the best subset of variables
best_subset = forward_selection_adj(train, idx_p, idx_t)
print("Best subset of variables (by index):", best_subset)

# Train a model using the selected subset of variables
X_train = train.iloc[:, best_subset]
y_train = train.iloc[:, idx_t]
X_train = sm.add_constant(X_train)  # Add constant for intercept
model_selected = sm.OLS(y_train, X_train).fit()

# Evaluate the model on the test set
X_test = test.iloc[:, best_subset]
y_test = test.iloc[:, idx_t]
X_test = sm.add_constant(X_test)  # Add constant for intercept
y_pred = model_selected.predict(X_test)

# Calculate generalization error (e.g., Mean Squared Error)
generalization_error_selected = mean_squared_error(y_test, y_pred)
print("Generalization error (selected variables):", generalization_error_selected)

# Train a model using all variables
X_train_all = train.iloc[:, idx_p]
X_train_all = sm.add_constant(X_train_all)
model_all = sm.OLS(y_train, X_train_all).fit()

# Evaluate the model with all variables on the test set
X_test_all = test.iloc[:, idx_p]
X_test_all = sm.add_constant(X_test_all)
y_pred_all = model_all.predict(X_test_all)

# Calculate generalization error for the model with all variables
generalization_error_all = mean_squared_error(y_test, y_pred_all)
print("Generalization error (all variables):", generalization_error_all)

# Compare the errors
if generalization_error_selected < generalization_error_all:
    print("The model with the selected subset of variables performs better.")
else:
    print("The model with all variables performs better.")

Best subset of variables (by index): [2, 10, 4, 3, 8, 1, 7, 9]
Generalization error (selected variables): 577.8556357100576
Generalization error (all variables): 580.5514936425963
The model with the selected subset of variables performs better.


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def forward_selection(train, idx_p, idx_t):
    selected_vars = []  # List to store selected variables
    non_selected_vars = idx_p.copy()  # Variables available for selection
    best_generalization_error = float('inf')  # Initialize the best generalization error
    stop = False  # Stopping criterion

    while non_selected_vars and not stop:
        best_var = None
        best_error = float('inf')

        for var in non_selected_vars:
            # Combine selected variables with the current variable
            current_vars = selected_vars + [var]

            # Split the training set into training and validation subsets
            train_subset, val_subset = train_test_split(train, test_size=0.25, random_state=42)

            # Prepare the data for regression
            X_train = train_subset.iloc[:, current_vars]
            y_train = train_subset.iloc[:, idx_t]
            X_train = sm.add_constant(X_train)  # Add constant for intercept

            X_val = val_subset.iloc[:, current_vars]
            y_val = val_subset.iloc[:, idx_t]
            X_val = sm.add_constant(X_val)  # Add constant for intercept

            # Fit the model
            model = sm.OLS(y_train, X_train).fit()

            # Predict on the validation set
            y_pred_val = model.predict(X_val)

            # Calculate generalization error (e.g., Mean Squared Error)
            error = mean_squared_error(y_val, y_pred_val)

            # Check if this variable improves the generalization error
            if error < best_error:
                best_error = error
                best_var = var

        # Check if the best error improves the overall generalization error
        if best_error < best_generalization_error:
            best_generalization_error = best_error
            selected_vars.append(best_var)
            non_selected_vars.remove(best_var)
        else:
            stop = True  # Stop if performance does not improve

    return selected_vars

## Polynomial regression

In [8]:
def add_polynomial_feature(data, idx_p, power):
    new_data = data.copy(deep = True)
    for i in range(0, len(idx_p)):
        for j in power:
            for k in range(2, j+1):
                new_data['{}_pow_{}'.format(new_data.columns[idx_p[i]],k)] = new_data.iloc[:,idx_p[i]]**k
    return(new_data)

In [9]:
train_poly = add_polynomial_feature(train, [8], [2])
print(train_poly)
# You can see a new column with values of x8^2
test_poly = add_polynomial_feature(test, [8], [2])
print(test_poly)
# Same for test

       y    x1    x2    x3  x4  x5  x6      x7      x8      x9  x10   x8_pow_2
108   81  19.6  25.1  27.2   3   4   4 -1.9284 -2.5712 -4.3301   57   6.611069
97    77  16.2  20.8  22.1   6   5   5 -0.6946 -2.0000 -1.3681   71   4.000000
5     80  17.7  19.8  18.3   6   6   7 -5.6382 -5.0000 -6.0000   94  25.000000
78    59  16.5  20.3  20.3   5   7   6 -4.3301 -5.3623 -4.5000   76  28.754261
11    72  18.3  19.6  19.4   7   5   6 -0.8682 -2.7362 -6.8944   90   7.486790
..   ...   ...   ...   ...  ..  ..  ..     ...     ...     ...  ...        ...
43    81  18.8  22.5  23.9   6   3   2  0.5209 -1.0000 -2.0000   72   1.000000
22    67  19.5  23.4  23.7   5   5   4 -1.5321 -3.0642 -0.8682   81   9.389322
72   159  25.0  33.5  35.5   1   1   1  1.0000  0.6946 -1.7101  166   0.482469
15    81  16.2  22.4  23.4   8   3   1  0.0000  0.3473 -2.5712  145   0.120617
40    88  16.9  20.3  20.7   6   6   5 -2.8191 -3.4641 -3.0000   92  11.999989

[88 rows x 12 columns]
       y    x1    x2    x3  

In [10]:
import statsmodels.api as sm

# Linear regression model using x8
X_train_linear = train_poly[['x8']]
X_train_linear = sm.add_constant(X_train_linear)  # Add constant for intercept
y_train = train_poly['y']
model_linear = sm.OLS(y_train, X_train_linear).fit()

# Non-linear regression model using x8 and x8^2
X_train_nonlinear = train_poly[['x8', 'x8_pow_2']]
X_train_nonlinear = sm.add_constant(X_train_nonlinear)  # Add constant for intercept
model_nonlinear = sm.OLS(y_train, X_train_nonlinear).fit()

# Print summaries of the models
print("Linear Model Summary:")
print(model_linear.summary())

print("\nNon-Linear Model Summary:")
print(model_nonlinear.summary())

Linear Model Summary:
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.191
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                     20.36
Date:                Fri, 11 Apr 2025   Prob (F-statistic):           2.02e-05
Time:                        12:57:40   Log-Likelihood:                -405.46
No. Observations:                  88   AIC:                             814.9
Df Residuals:                      86   BIC:                             819.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         94.3075      3.0